<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2024-07-03 17:07:45--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2024-07-03 17:07:45--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2024-07-03 17:07:45--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.

In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['nirvana.txt',
 'rihanna.txt',
 'ludacris.txt',
 'beatles.txt',
 'bjork.txt',
 'patti-smith.txt',
 'bruno-mars.txt',
 'kanye.txt',
 'jimi-hendrix.txt',
 'nursery_rhymes.txt',
 'britney-spears.txt',
 'missy-elliott.txt',
 'prince.txt',
 'drake.txt',
 'al-green.txt',
 'leonard-cohen.txt',
 'nicki-minaj.txt',
 'johnny-cash.txt',
 'adele.txt',
 'nickelback.txt',
 'paul-simon.txt',
 'Lil_Wayne.txt',
 'cake.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'Kanye_West.txt',
 'notorious_big.txt',
 'bob-marley.txt',
 'amy-winehouse.txt',
 'lin-manuel-miranda.txt',
 'alicia-keys.txt',
 'dj-khaled.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'notorious-big.txt',
 'blink-182.txt',
 'bieber.txt',
 'dolly-parton.txt',
 'bruce-springsteen.txt',
 'radiohead.txt',
 'janisjoplin.txt',
 'lorde.txt',
 'eminem.txt',
 'joni-mitchell.txt',
 'michael-jackson.txt',
 'disney.txt',
 'bob-dylan.txt',
 'kanye-west.txt',
 'dickinson.txt']

In [26]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)
df.head()

<ipython-input-26-89d9bc44a57b>:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,"""Don't worry about a thing,"
1,'Cause every little thing gonna be all right.
2,"Singin': ""Don't worry about a thing,"
3,"'Cause every little thing gonna be all right!""..."
4,"Smiled with the risin' sun,"


In [27]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2218


### 1 - Preprocesamiento

In [28]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [29]:
# Demos un vistazo
sentence_tokens[:5]

[["don't", 'worry', 'about', 'a', 'thing'],
 ["'cause", 'every', 'little', 'thing', 'gonna', 'be', 'all', 'right'],
 ["singin'", "don't", 'worry', 'about', 'a', 'thing'],
 ["'cause",
  'every',
  'little',
  'thing',
  'gonna',
  'be',
  'all',
  'right',
  'rise',
  'up',
  'this',
  "mornin'"],
 ['smiled', 'with', 'the', "risin'", 'sun']]

### 2 - Crear los vectores (word2vec)

In [30]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [31]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [32]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [33]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2218


In [34]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 532


### 3 - Entrenar embeddings

In [37]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 45291.828125
Loss after epoch 1: 45959.203125
Loss after epoch 2: 46339.140625
Loss after epoch 3: 46199.765625
Loss after epoch 4: 45757.359375
Loss after epoch 5: 46127.546875
Loss after epoch 6: 45864.6875
Loss after epoch 7: 46130.0625
Loss after epoch 8: 45413.71875
Loss after epoch 9: 45067.0625
Loss after epoch 10: 44103.9375
Loss after epoch 11: 44619.25
Loss after epoch 12: 43089.1875
Loss after epoch 13: 43893.6875
Loss after epoch 14: 43731.0625
Loss after epoch 15: 43316.875
Loss after epoch 16: 43519.75
Loss after epoch 17: 44251.3125
Loss after epoch 18: 43417.6875
Loss after epoch 19: 43808.125


(217638, 359260)

### 4 - Ensayar

In [38]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('shed', 0.7492054104804993),
 ('calling', 0.7074381113052368),
 ("c'mon", 0.657069981098175),
 ('understand', 0.6355472803115845),
 ("darlin'", 0.6094400882720947),
 ('tears', 0.595602810382843),
 ('given', 0.5941136479377747),
 ('satisfy', 0.571770429611206),
 ('believe', 0.5668187141418457),
 ('na', 0.5652367472648621)]

In [39]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('learn', -0.03768162056803703),
 ('as', -0.050765518099069595),
 ('upsetting', -0.07124830782413483),
 ('babylon', -0.07131063938140869),
 ('these', -0.07201641798019409),
 ('when', -0.07406779378652573),
 ("y'all", -0.07896115630865097),
 ('we', -0.07996365427970886),
 ('bob', -0.08414139598608017),
 ('yeah', -0.08460512012243271)]

In [40]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["babylon"], topn=10)

[('throne', 0.7646043300628662),
 ('leaving', 0.6807588338851929),
 ('system', 0.5816575288772583),
 ('blowing', 0.5084211230278015),
 ('moving', 0.5072095990180969),
 ('today', 0.5054658055305481),
 ('air', 0.4988565146923065),
 ('cok', 0.49800240993499756),
 ('su', 0.48785555362701416),
 ("y'all", 0.4860841631889343)]

In [42]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["three"], topn=10)

[('birds', 0.6516872048377991),
 ('arrival', 0.5262256860733032),
 ('two', 0.512079119682312),
 ('word', 0.494046688079834),
 ('movement', 0.47712400555610657),
 ('fighting', 0.47653234004974365),
 ('door', 0.47446534037590027),
 ('wait', 0.46904876828193665),
 ('na', 0.46458208560943604),
 ("darlin'", 0.45063039660453796)]

In [46]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["africa"], topn=5)

[('unite', 0.8263668417930603),
 ('brought', 0.5953739881515503),
 ('po', 0.5838794708251953),
 ('yard', 0.5636486411094666),
 ('america', 0.5587740540504456)]

In [18]:
# Ensayar con una palabra que no está en el vocabulario:
# w2v_model.wv.most_similar(negative=["diedaa"])

In [51]:
# el método `get_vector` permite obtener los vectores:
vector_africa = w2v_model.wv.get_vector("africa")
print(vector_africa)

[-0.133961    0.08378483 -0.18624732  0.49558687 -0.04461044  0.02597883
 -0.39289337  0.12281882 -0.25316957  0.11156434  0.15068887  0.36497238
 -0.01354092  0.2624582  -0.37540573 -0.00942466  0.14073029  0.4262139
  0.0334273  -0.2972196  -0.40289226 -0.03191782 -0.06124923  0.42154256
 -0.33600116 -0.35953984 -0.30476865 -0.289576    0.51650435  0.11034039
  0.13764746 -0.30511713  0.26263767 -0.17866074 -0.10495462  0.40942574
  0.17890333 -0.41941127 -0.01067857  0.15039648 -0.02345174 -0.0539102
  0.05370225 -0.11934365  0.14101657  0.06452571 -0.19534723 -0.16397652
  0.30996552 -0.11147574  0.1552969   0.16575809  0.21815722  0.03471778
  0.308392   -0.30794522  0.35213286 -0.09894267 -0.07417348 -0.35638222
  0.17391154 -0.37922716 -0.19596246 -0.15171856 -0.5358688   0.05120009
  0.48432463 -0.10713302  0.00279433 -0.18515128 -0.19220524 -0.05129953
  0.06144339  0.28596795  0.62000185 -0.09604405 -0.09532335 -0.16652215
 -0.01857482  0.03065601  0.14784351 -0.26051423 -0.0

In [52]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_africa)

[('africa', 1.0),
 ('unite', 0.8263668417930603),
 ('brought', 0.5953739881515503),
 ('po', 0.5838794708251953),
 ('yard', 0.5636486411094666),
 ('america', 0.5587740540504456),
 ('na', 0.5022343993186951),
 ('better', 0.4938502907752991),
 ('ready', 0.4893672466278076),
 ("jammin'", 0.48159074783325195)]

In [57]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["mystic"], topn=10)

[('natural', 0.9759191274642944),
 ("there's", 0.7724676132202148),
 ('mob', 0.7284840941429138),
 ('angry', 0.7092230916023254),
 ('rammer', 0.7056198120117188),
 ('shakedown', 0.6250330805778503),
 ('hammer', 0.6225574612617493),
 ('dreadlock', 0.6197691559791565),
 ('satisfy', 0.6149569153785706),
 ('jelly', 0.603087306022644)]

### 5 - Visualizar agrupación de vectores

In [58]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [59]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [60]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [61]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.